In [1]:
### Import Statements ###
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFE
import seaborn as sns
#import missingno as ms
%matplotlib inline

In [3]:
df = pd.read_csv('criminal_train.csv')
df.head()

,PERID,IFATHER,NRCH17_2,IRHHSIZ2,IIHHSIZ2,IRKI17_2,IIKI17_2,IRHH65_2,IIHH65_2,PRXRETRY,...,TOOLONG,TROUBUND,PDEN10,COUTYP2,MAIIN102,AIIND102,ANALWT_C,VESTR,VEREP,Criminal
0,25095143,4,2,4,1,3,1,1,1,99,...,1,2,1,1,2,2,3884.805998,40026,1,0
1,13005143,4,1,3,1,2,1,1,1,99,...,2,2,2,3,2,2,1627.108106,40015,2,1
2,67415143,4,1,2,1,2,1,1,1,99,...,2,2,2,3,2,2,4344.957980,40024,1,0
3,70925143,4,0,2,1,1,1,1,1,99,...,2,2,1,1,2,2,792.521931,40027,1,0
4,75235143,1,0,6,1,4,1,1,1,99,...,2,2,2,2,2,2,1518.118526,40001,2,0


In [13]:
'''from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df)
scaled_features = scaler.transform(df)
df_feat = pd.DataFrame(scaled_features,columns=df.columns)
df_feat.head()'''

'from sklearn.preprocessing import StandardScaler\nscaler = StandardScaler()\nscaler.fit(df)\nscaled_features = scaler.transform(df)\ndf_feat = pd.DataFrame(scaled_features,columns=df.columns)\ndf_feat.head()'

In [14]:
'''from sklearn.decomposition import PCA
pca = PCA(n_components=50)
pca.fit(scaled_data)
x_pca = pd.DataFrame(pca.transform(scaled_data))
'''

'from sklearn.decomposition import PCA\npca = PCA(n_components=50)\npca.fit(scaled_data)\nx_pca = pd.DataFrame(pca.transform(scaled_data))\n'

In [3]:
x=df.drop('Criminal',axis=1)
y=df['Criminal']

In [4]:
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier,GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import confusion_matrix,precision_score

In [ ]:
# df[df < 0] = 0
# test = SelectKBest(score_func=chi2, k=60)
# fit = test.fit(df.drop('Criminal',axis=1),df['Criminal'])
# #print(len(fit.scores_))
# features = fit.transform(df.drop('Criminal',axis=1))
# # summarize selected features
# print(features.shape)
# dfy = pd.DataFrame(data=features,columns=range(1,61))
# dfyc = pd.DataFrame(df['Criminal'],columns=['61'])
# dfy = pd.concat([dfy,df['Criminal']],axis=1)

In [5]:
model = RandomForestClassifier()
parameters={'n_estimators':[450],'min_samples_split':[3]} # for rf
#parameters = {'penalty':['l1'],'max_iter':[200],} # for logistic
#parameters = {'n_estimators':[90],'max_features':['sqrt'],'min_samples_split':[3]} #for extratrees
#parameters={'n_estimators':[80],'max_depth':[2]} #gradientboosting
#parameters={'n_estimators':[90],'gamma':[0.1]}

In [6]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2,random_state=101)

In [7]:
best_model = GridSearchCV(model,parameters,n_jobs=5,verbose=True)
best_model.fit(X_train,y_train)
best_model.best_params_

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=5)]: Done   3 out of   3 | elapsed:   24.0s finished


{'min_samples_split': 3, 'n_estimators': 450}

In [8]:
y_predict = best_model.predict(X_test)
print(confusion_matrix(y_test,y_predict))
precision_score(y_test,y_predict)

[[10423   156]
 [  229   533]]


0.7735849056603774

In [9]:
dft = pd.read_csv('criminal_test.csv')
dft. head()

,PERID,IFATHER,NRCH17_2,IRHHSIZ2,IIHHSIZ2,IRKI17_2,IIKI17_2,IRHH65_2,IIHH65_2,PRXRETRY,...,POVERTY3,TOOLONG,TROUBUND,PDEN10,COUTYP2,MAIIN102,AIIND102,ANALWT_C,VESTR,VEREP
0,66583679,4,0,4,1,2,1,1,1,99,...,2,2,2,1,1,2,2,16346.795400,40020,1
1,35494679,4,0,4,1,1,1,1,1,99,...,3,2,2,1,1,2,2,3008.863906,40044,2
2,79424679,2,0,3,1,2,1,1,1,99,...,1,2,2,2,2,2,2,266.952503,40040,2
3,11744679,4,0,6,1,2,1,1,1,99,...,3,2,2,1,1,2,2,5386.928199,40017,1
4,31554679,1,0,4,1,3,1,1,1,99,...,3,2,1,1,1,2,2,173.489895,40017,1


In [38]:
# dft[dft < 0] = 0
# test = SelectKBest(score_func=chi2, k=60)
# #print(len(fit.scores_))
# features = fit.transform(dft)
# # summarize selected features
# print(features.shape)
# dfyt = pd.DataFrame(data=features,columns=range(1,61))

In [17]:
final_predict = pd.DataFrame(best_model.predict(dft),columns=['Criminal'])
#result = pd.DataFrame(dft['PERID'],columns=['PERID'])
result = pd.concat([result,final_predict],axis=1)
result.to_csv("response.csv",index=False,header=['PERID','Criminal'])

NameError: name 'dft' is not defined

In [ ]:
result['Criminal'].value_counts()